In [1]:
import math

import numpy as np
import tensorflow as tf

In [7]:
tf.reset_default_graph()
def get_mean_std(x: tf.Tensor):
    mean = tf.reduce_mean(x, axis=-1, keepdims=True)
    squared = tf.square(x - mean)
    variance = tf.reduce_mean(squared, axis=-1, keepdims=True)
    std = tf.sqrt(variance)

    return mean, std

def layer_norm(layer: tf.Tensor):
    with tf.variable_scope("norm"):
        scale = tf.get_variable("scale", shape=layer.shape[-1], dtype=tf.float32)
        base = tf.get_variable("base", shape=layer.shape[-1], dtype=tf.float32)
        mean, std = get_mean_std(layer)
        norm = (layer - mean) / (std + 1e-6)
        return norm * scale + base

def attention(query: tf.Tensor, key: tf.Tensor, value: tf.Tensor, *,
              mask: tf.Tensor,
              keep_prob: float):
    d_k = query.shape[-1].value
    scores = tf.matmul(query, tf.transpose(key, perm=[0, 1, 3, 2]))
    scores = scores / tf.constant(math.sqrt(d_k))
    mask_add = ((scores * 0) - 1e9) * (tf.constant(1.) - mask)
    scores = scores * mask + mask_add
    attn = tf.nn.softmax(scores, axis=-1)
    attn = tf.nn.dropout(attn, keep_prob)
    return tf.matmul(attn, value), attn

def prepare_for_multi_head_attention(x: tf.Tensor, heads: int, name: str):
    n_batches, seq_len, d_model = x.shape
    assert d_model % heads == 0
    d_k = d_model // heads
    x = tf.layers.dense(x, units=d_model, name=name)
    x = tf.reshape(x, shape=[n_batches, seq_len, heads, d_k])
    x = tf.transpose(x, perm=[0, 2, 1, 3])

    return x

def multi_head_attention(query: tf.Tensor, key: tf.Tensor, value: tf.Tensor, *,
                         mask: tf.Tensor,
                         heads: int,
                         keep_prob: float):
    with tf.variable_scope("multi_head"):
        n_batches, seq_len, d_model = query.shape
        query = prepare_for_multi_head_attention(query, heads, "query")
        key = prepare_for_multi_head_attention(key, heads, "key")
        value = prepare_for_multi_head_attention(value, heads, "value")
        mask = tf.expand_dims(mask, axis=1)
        out, _ = attention(query, key, value, mask=mask, keep_prob=keep_prob)
        out = tf.transpose(out, perm=[0, 2, 1, 3])
        out = tf.reshape(out, shape=[n_batches, seq_len, d_model])
        return tf.layers.dense(out, units=d_model, name="attention")
    
def feed_forward(x: tf.Tensor,
                 d_model: int, d_ff: int, keep_prob: float):
    
    with tf.variable_scope("feed_forward"):
        hidden = tf.layers.dense(x, units=d_ff, name="hidden")
        hidden = tf.nn.relu(hidden)
        hidden = tf.nn.dropout(hidden, keep_prob=keep_prob)
        return tf.layers.dense(hidden, units=d_model, name="out")
    
def encoder_layer(x: tf.Tensor, *,
                  mask: tf.Tensor, index: int, heads: int,
                  keep_prob: float, d_ff: int):
    d_model = x.shape[-1]

    with tf.variable_scope(f"attention_{index}"):
        attention_out = multi_head_attention(x, x, x,
                                             mask=mask, heads=heads, keep_prob=keep_prob)
        added = x + tf.nn.dropout(attention_out, keep_prob)
        x = layer_norm(added)
    with tf.variable_scope(f"ff_{index}"):
        ff_out = feed_forward(x, d_model, d_ff, keep_prob)
        added = x + tf.nn.dropout(ff_out, keep_prob)
        return layer_norm(added)

def encoder(x: tf.Tensor, *,
            mask: tf.Tensor,
            n_layers: int,
            heads: int, keep_prob: float, d_ff: int):
    with tf.variable_scope("encoder"):
        for i in range(n_layers):
            x = encoder_layer(x,
                              mask=mask, index=i,
                              heads=heads, keep_prob=keep_prob, d_ff=d_ff)

        return x
    
def decoder_layer(encoding: tf.Tensor, x: tf.Tensor, *,
                  enc_mask: tf.Tensor, mask: tf.Tensor,
                  index: int, heads: int, keep_prob: float, d_ff: int):
    d_model = encoding.shape[-1]
    
    with tf.variable_scope(f"{index}_self_attention"):
        attention_out = multi_head_attention(x, x, x,
                                             mask=mask, heads=heads, keep_prob=keep_prob)
        added = x + tf.nn.dropout(attention_out, keep_prob=keep_prob)
        x = layer_norm(added)
    with tf.variable_scope(f"{index}_encoding_attention"):
        attention_out = multi_head_attention(x, encoding, encoding,
                                             mask=enc_mask, heads=heads, keep_prob=keep_prob)
        
        added = x + tf.nn.dropout(attention_out, keep_prob=keep_prob)
        x = layer_norm(added)
    with tf.variable_scope(f"{index}_ff"):
        ff_out = feed_forward(x, d_model, d_ff, keep_prob)
        
        added = x + tf.nn.dropout(ff_out, keep_prob)
        return layer_norm(added)

def decoder(encoding: tf.Tensor, x: tf.Tensor, *,
            enc_mask: tf.Tensor, mask: tf.Tensor,
            n_layers: int,
            heads: int, keep_prob: float, d_ff: int):
    with tf.variable_scope("decoder"):
        for i in range(n_layers):
            x = decoder_layer(encoding, x,
                              enc_mask=enc_mask, mask=mask, index=i,
                              heads=heads, keep_prob=keep_prob, d_ff=d_ff)

        return x
    
def get_embeddings(input_ids: tf.Tensor, output_ids: tf.Tensor,
                   vocab_size: int, d_model: int):

    word_embeddings = tf.get_variable("word_embeddings",
                                      shape=[vocab_size, d_model],
                                      dtype=tf.float32,
                                      initializer=tf.initializers.random_normal())
    in_emb = tf.nn.embedding_lookup(word_embeddings, input_ids)
    out_emb = tf.nn.embedding_lookup(word_embeddings, output_ids)
    return word_embeddings, in_emb, out_emb

def generate_positional_encodings(d_model: int, max_len: int = 5000):
    encodings = np.zeros((max_len, d_model), dtype=float)
    position = np.arange(0, max_len).reshape((max_len, 1))
    two_i = np.arange(0, d_model, 2)
    div_term = np.exp(-math.log(10000.0) * two_i / d_model)
    encodings[:, 0::2] = np.sin(position * div_term)
    encodings[:, 1::2] = np.cos(position * div_term)
    return tf.constant(encodings.reshape((1, max_len, d_model)),
                       dtype=tf.float32, name="positional_encodings")

def prepare_embeddings(x: tf.Tensor, *,
                       positional_encodings: tf.Tensor,
                       keep_prob: float, is_input: bool):
    name = "prepare_input" if is_input else "prepare_output"
    with tf.variable_scope(name):
        _, seq_len, _ = x.shape
        x = x + positional_encodings[:, :seq_len, :]
        x = tf.nn.dropout(x, keep_prob)
        return layer_norm(x)

def generator(x: tf.Tensor, *, vocab_size: int):
#
    res = tf.layers.dense(x, units=vocab_size, name="generator")
    return tf.nn.log_softmax(res, axis=-1)


def label_smoothing_loss(results: tf.Tensor, expected: tf.Tensor, *,
                         vocab_size: int, smoothing: float):
    results = tf.reshape(results, shape=(-1, vocab_size))
    expected = tf.reshape(expected, shape=[-1])

    confidence = 1 - smoothing
    smoothing = smoothing / (vocab_size - 1)
    expected = tf.one_hot(expected, depth=vocab_size) * (confidence - smoothing)
    expected += smoothing

    results = tf.distributions.Categorical(logits=results)
    expected = tf.distributions.Categorical(logits=expected)
    return tf.reduce_mean(tf.distributions.kl_divergence(results, expected))

def generate_data(batch_size: int, seq_len: int, vocab_size: int):
    start_token = vocab_size - 1
    repeat_token = vocab_size - 2
    vocab_size -= 2

    inputs = np.random.randint(0, vocab_size, size=(batch_size, seq_len))

    outputs = np.zeros((batch_size, seq_len + 1), dtype=int)
    outputs[:, 1:] = np.flip(inputs, 1)
    outputs[:, 0] = start_token

    for i in range(batch_size):
        v = np.zeros(vocab_size, dtype=bool)
        for j in range(seq_len):
            word = inputs[i, j]
            if v[word]:
                v[word] = False
                outputs[i][seq_len - j] = repeat_token
            else:
                v[word] = True

    return inputs, outputs
#     return inputs, inputs

def noam_learning_rate(step: int, warm_up: float, d_model: int):
    return (d_model ** -.5) * min(step ** -.5, step * warm_up ** -1.5)

def output_subsequent_mask(seq_len: int):
    mask = np.zeros((seq_len, seq_len), dtype=float)
    for i in range(seq_len):
        for j in range(i + 1):
            mask[i, j] = 1.

    return mask

def train():
    seq_length = 10
    vocab_size = 10 + 1 + 1
    vocab_str = [f"{i}" for i in range(10)]
    vocab_str += ['X', 'S']
    
    batch_size = 32  # 12000
#     d_model = 128  # 512
    d_model = 512
    heads = 8
    keep_prob = 0.90
#     n_layers = 2  # 6
    n_layers = 6
#     d_ff = 256  # 2048
    d_ff = 2048
    positional_encodings = generate_positional_encodings(d_model)
    inputs = tf.placeholder(dtype=tf.int32,
                            shape=(batch_size, seq_length), name="input")
    outputs = tf.placeholder(dtype=tf.int32,
                             shape=(batch_size, seq_length), name="output")
    expected = tf.placeholder(dtype=tf.int32,
                              shape=(batch_size, seq_length), name="expected")
    inputs_mask = tf.placeholder(dtype=tf.float32,
                                 shape=(1, 1, seq_length),
                                 name="input_mask")
    output_mask = tf.placeholder(dtype=tf.float32,
                                 shape=(1, seq_length, seq_length),
                                 name="output_mask")
    learning_rate = tf.placeholder(dtype=tf.float32, name="learning_rate")
    w_embed, input_embeddings, output_embeddings = get_embeddings(inputs, outputs, vocab_size,
                                                                  d_model)
    input_embeddings = prepare_embeddings(input_embeddings,
                                          positional_encodings=positional_encodings,
                                          keep_prob=keep_prob,
                                          is_input=True)
    output_embeddings = prepare_embeddings(output_embeddings,
                                           positional_encodings=positional_encodings,
                                           keep_prob=keep_prob,
                                           is_input=False)

    encoding = encoder(input_embeddings, mask=inputs_mask, n_layers=n_layers, heads=heads,
                       keep_prob=keep_prob, d_ff=d_ff)
    decoding = decoder(encoding, output_embeddings,
                       enc_mask=inputs_mask, mask=output_mask,
                       n_layers=n_layers, heads=heads, keep_prob=keep_prob, d_ff=d_ff)
    log_results = generator(decoding, vocab_size=vocab_size)
    results = tf.exp(log_results)
    loss = label_smoothing_loss(log_results, expected, vocab_size=vocab_size, smoothing=0.0)
    adam = tf.train.AdamOptimizer(learning_rate=learning_rate, epsilon=1e-5)
    params = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, params), 5.)
    grads_and_vars = list(zip(grads, params))
    train_op = adam.apply_gradients(grads_and_vars, name="apply_gradients")

    warm_up = 400
    batch_in_mask = np.ones((1, 1, seq_length), dtype=float)
    batch_out_mask = output_subsequent_mask(seq_length)
    batch_out_mask = batch_out_mask.reshape(1, seq_length, seq_length)
    def __print_seq(seq):
        return ' '.join([vocab_str[i] for i in seq])

    return
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())

        for i in range(1000):
            lr = noam_learning_rate(i + 1, warm_up, d_model)
            
            batch_in, batch_out = generate_data(batch_size, seq_length, vocab_size)
            _, batch_loss, batch_res = session.run([train_op, loss, results],
                                                   feed_dict={
                                                       learning_rate: lr,
                                                       inputs: batch_in,
                                                       outputs: batch_out[:, :-1],
                                                       expected: batch_out[:, 1:],
                                                       inputs_mask: batch_in_mask,
                                                       output_mask: batch_out_mask
                                                   })
            if i % 100 == 0:
                print(f"step={i}\tloss={batch_loss: .6f}")
                print(f"inp=  {__print_seq(batch_in[0])}")
                print(f"exp={__print_seq(batch_out[0])}")
                print(f"res=  {__print_seq(np.argmax(batch_res[0], -1))}")
                
                
                
# generate_data(1, 10, 6)[1]#[:, :-1]

In [8]:
train()

In [9]:
def count_trainable_params():
    """Count the total trainable parameters. """

    total_params = [np.prod(v.get_shape().as_list()) \
                    for v in tf.trainable_variables()]

    return np.sum(total_params)
    return total_params

In [10]:
count_trainable_params()

44152844

In [20]:
l = "0.000000000000000000e+00 7.660613250732421875e+01 4.866783905029296875e+01 6.256052398681640625e+01 1.768428344726562500e+02 2.817915039062500000e+02 4.009801330566406250e+02 4.420607299804687500e+02 4.172068786621093750e+02 4.315484008789062500e+02 5.542190551757812500e+02 6.532561645507812500e+02 7.802635498046875000e+02 8.404612426757812500e+02 8.039700317382812500e+02 8.176761474609375000e+02 9.435902099609375000e+02 9.704349365234375000e+02 1.091109497070312500e+03 1.182813842773437500e+03 1.112384887695312500e+03 1.005608398437500000e+03 9.735283813476562500e+02 9.477186279296875000e+02 9.282864990234375000e+02 8.047166748046875000e+02 7.595912475585937500e+02 6.392043457031250000e+02 5.394482421875000000e+02 5.829269409179687500e+02 5.232661132812500000e+02 4.188103027343750000e+02 2.809184875488281250e+02 1.833175659179687500e+02 9.879071807861328125e+01 1.603836975097656250e+02 1.157822875976562500e+02 6.693618774414062500e+00 -1.286782989501953125e+02 -2.346667785644531250e+02 -3.034371643066406250e+02 -2.330605316162109375e+02 -2.563178405761718750e+02 -3.719937133789062500e+02 -5.021560668945312500e+02 -6.053215332031250000e+02 -6.474117431640625000e+02 -5.456500244140625000e+02 -5.512614135742187500e+02 -6.777222290039062500e+02 -7.904136962890625000e+02 -8.541801757812500000e+02 -8.560670776367187500e+02 -7.148131103515625000e+02 -6.990165405273437500e+02 -8.010830078125000000e+02 -9.357318115234375000e+02 -9.596210937500000000e+02 -8.687566528320312500e+02 -7.234572143554687500e+02 -6.419649047851562500e+02 -6.411007080078125000e+02 -7.802884521484375000e+02 -7.993125000000000000e+02 -6.903040771484375000e+02 -5.863652343750000000e+02 -4.723192749023437500e+02 -4.369258422851562500e+02 -5.590545654296875000e+02 -5.900598754882812500e+02 -5.087196655273437500e+02 -4.257839965820312500e+02 -3.047027587890625000e+02 -2.447705993652343750e+02 -3.509026489257812500e+02 -3.768998413085937500e+02 -2.917509460449218750e+02 -2.096071777343750000e+02 -9.005464935302734375e+01 -4.071420288085937500e+01 -1.522337646484375000e+02 -1.829584045410156250e+02 -1.185323638916015625e+02 -3.879616546630859375e+01 7.331315612792968750e+01 1.103220520019531250e+02 -3.668487548828125000e+00 -2.317793273925781250e+01 5.886492156982421875e+01 1.407001037597656250e+02 2.443132476806640625e+02 2.778391418457031250e+02 1.679980163574218750e+02 1.644577178955078125e+02 2.637382812500000000e+02 3.392546691894531250e+02 4.387383422851562500e+02 4.747561645507812500e+02 3.763174133300781250e+02 3.853419189453125000e+02 4.968876037597656250e+02 5.541459960937500000e+02 6.677280273437500000e+02 7.292206420898437500e+02 6.461187133789062500e+02 6.315888061523437500e+02 6.850690307617187500e+02 7.026297607421875000e+02 8.229422607421875000e+02 9.338488769531250000e+02 8.877816772460937500e+02 8.537188110351562500e+02 8.705288085937500000e+02 8.674578247070312500e+02 9.854487915039062500e+02 1.106352050781250000e+03 1.086542968750000000e+03 1.029567749023437500e+03 1.041811645507812500e+03 1.023564758300781250e+03 1.115880737304687500e+03 1.231065063476562500e+03 1.232346435546875000e+03 1.123647216796875000e+03 1.026800170898437500e+03 9.958764038085937500e+02 9.958735961914062500e+02 8.925708007812500000e+02 8.705059204101562500e+02 7.457762451171875000e+02 6.347423095703125000e+02 6.181170654296875000e+02 5.742575073242187500e+02 4.330846557617187500e+02 3.558571166992187500e+02 2.257972564697265625e+02 1.267504653930664062e+02 1.032160720825195312e+02 5.238403701782226562e+01 -9.037109375000000000e+01 -1.639448089599609375e+02 -2.955775756835937500e+02 -3.872927246093750000e+02 -3.994508972167968750e+02 -4.361227416992187500e+02 -5.764782104492187500e+02 -6.394445800781250000e+02 -7.701058349609375000e+02 -8.658799438476562500e+02 -8.398657226562500000e+02 -8.607933959960937500e+02 -9.860188598632812500e+02 -1.043102905273437500e+03 -1.159798583984375000e+03 -1.263801147460937500e+03 -1.217075683593750000e+03 -1.275475830078125000e+03 -1.396523315429687500e+03 -1.432555419921875000e+03 -1.482848022460937500e+03 -1.549498413085937500e+03 -1.456521240234375000e+03 -1.515053100585937500e+03 -1.656388916015625000e+03 -1.726985229492187500e+03 -1.704110839843750000e+03 -1.614791137695312500e+03 -1.502876586914062500e+03 -1.512252563476562500e+03 -1.567085693359375000e+03 -1.661379882812500000e+03 -1.608913085937500000e+03 -1.466566284179687500e+03 -1.385791503906250000e+03 -1.368803710937500000e+03 -1.318784179687500000e+03 -1.384758300781250000e+03 -1.302908935546875000e+03 -1.158460327148437500e+03 -1.114336181640625000e+03 -1.110932006835937500e+03 -1.110737792968750000e+03 -1.171537353515625000e+03 -1.085802978515625000e+03 -9.431120605468750000e+02 -9.145984497070312500e+02 -9.302151489257812500e+02 -9.503737792968750000e+02 -1.000595825195312500e+03 -9.076758422851562500e+02 -7.676271972656250000e+02 -7.381824951171875000e+02 -7.603186645507812500e+02 -7.904353027343750000e+02 -8.375729370117187500e+02 -7.427922363281250000e+02 -6.127117919921875000e+02 -5.727778320312500000e+02 -5.753777465820312500e+02 -6.503439331054687500e+02 -7.246762695312500000e+02 -6.509824829101562500e+02 -5.203887329101562500e+02 -4.290184020996093750e+02 -3.428137512207031250e+02 -3.666144714355468750e+02 -5.116567993164062500e+02 -6.016269531250000000e+02 -5.664926147460937500e+02 -4.375774536132812500e+02 -3.507029418945312500e+02 -3.769093933105468750e+02 -5.008746643066406250e+02 -6.153374023437500000e+02 -6.469715576171875000e+02 -5.289004516601562500e+02 -5.036053466796875000e+02 -6.028904418945312500e+02 -7.396674194335937500e+02 -8.396035766601562500e+02 -8.755225830078125000e+02 -7.539934082031250000e+02 -7.398668212890625000e+02 -8.424281005859375000e+02 -9.803550415039062500e+02 -1.076890014648437500e+03 -1.089615966796875000e+03 -9.530034179687500000e+02 -9.186844482421875000e+02 -1.015243347167968750e+03 -1.154182006835937500e+03 -1.246245605468750000e+03 -1.214258666992187500e+03 -1.063668579101562500e+03 -1.001302856445312500e+03 -1.074882446289062500e+03 -1.221111694335937500e+03 -1.301789062500000000e+03 -1.250005859375000000e+03 -1.098208740234375000e+03 -1.030892700195312500e+03 -1.099010986328125000e+03 -1.248632080078125000e+03 -1.318450805664062500e+03 -1.255412597656250000e+03 -1.104279785156250000e+03 -1.036392456054687500e+03 -1.103878662109375000e+03 -1.254471923828125000e+03 -1.318770019531250000e+03 -1.249334228515625000e+03 -1.099882812500000000e+03 -1.026240356445312500e+03 -1.087010498046875000e+03 -1.238675659179687500e+03 -1.301606445312500000e+03 -1.228653564453125000e+03 -1.083265502929687500e+03 -1.000697021484375000e+03 -1.048800048828125000e+03 -1.201036499023437500e+03 -1.260617675781250000e+03 -1.184663085937500000e+03 -1.044682617187500000e+03 -9.545008544921875000e+02 -9.889220581054687500e+02 -1.139652709960937500e+03 -1.195468261718750000e+03 -1.116760742187500000e+03 -9.825870971679687500e+02 -8.853644409179687500e+02 -9.047306518554687500e+02 -1.052101806640625000e+03 -1.103994995117187500e+03 -1.024366333007812500e+03 -8.991753540039062500e+02 -7.940134277343750000e+02 -8.008631591796875000e+02 -9.442029418945312500e+02 -9.985795898437500000e+02 -9.290281982421875000e+02 -8.153121948242187500e+02 -7.006688232421875000e+02 -6.900464477539062500e+02 -8.264132690429687500e+02 -8.877943115234375000e+02 -8.377745361328125000e+02 -7.379658203125000000e+02 -6.140783691406250000e+02 -5.797177124023437500e+02 -7.042103881835937500e+02 -7.729326171875000000e+02 -7.359506835937500000e+02 -6.478305664062500000e+02 -5.172448730468750000e+02 -4.583356933593750000e+02 -5.633289794921875000e+02 -6.287738647460937500e+02 -5.860895996093750000e+02 -5.077944335937500000e+02 -3.754016113281250000e+02 -3.070396118164062500e+02 -3.947136840820312500e+02 -4.518736572265625000e+02 -4.047741699218750000e+02 -3.353129882812500000e+02 -2.029303588867187500e+02 -1.254740600585937500e+02 -2.105492553710937500e+02 -2.623100585937500000e+02 -2.137001953125000000e+02 -1.474293212890625000e+02 -1.583160400390625000e+01 5.906323242187500000e+01 -3.445892333984375000e+01 -9.752331542968750000e+01 -6.839282226562500000e+01 -4.387084960937500000e+00 1.282836914062500000e+02 2.073006591796875000e+02 1.182342529296875000e+02 4.574206542968750000e+01 5.661010742187500000e+01 1.160734863281250000e+02 2.486966552734375000e+02 3.326252441406250000e+02 2.483150634765625000e+02 1.603890380859375000e+02 1.431221923828125000e+02 1.946531982421875000e+02 3.227423095703125000e+02 4.250677490234375000e+02 3.625954589843750000e+02 2.446257324218750000e+02 1.635668945312500000e+02 2.101029052734375000e+02 3.158576660156250000e+02 4.094730224609375000e+02 3.667976074218750000e+02 2.392652587890625000e+02 1.422540283203125000e+02 1.949921875000000000e+02 2.768945312500000000e+02 3.390408935546875000e+02 3.030329589843750000e+02 1.733438720703125000e+02 7.160510253906250000e+01 1.296287841796875000e+02 1.871085205078125000e+02 2.544422607421875000e+02 2.359082031250000000e+02 1.156817626953125000e+02 1.065032958984375000e+01 6.920910644531250000e+01 7.849975585937500000e+01 -8.210327148437500000e+00 -6.339843750000000000e+01 -1.821607055664062500e+02 -2.727229614257812500e+02 -1.956645507812500000e+02 -1.930780029296875000e+02 -3.144580688476562500e+02 -4.054708862304687500e+02 -5.278016357421875000e+02 -6.047639770507812500e+02 -5.154212036132812500e+02 -4.996386108398437500e+02 -5.996979980468750000e+02 -6.764913940429687500e+02 -8.017155151367187500e+02 -8.896154174804687500e+02 -8.141918945312500000e+02 -8.066560058593750000e+02 -8.802751464843750000e+02 -9.325632324218750000e+02 -1.047397949218750000e+03 -1.145041137695312500e+03 -1.076757324218750000e+03 -1.102380249023437500e+03 -1.125327148437500000e+03 -1.136742797851562500e+03 -1.202004516601562500e+03 -1.257374511718750000e+03 -1.173946289062500000e+03 -1.227710693359375000e+03 -1.314135620117187500e+03 -1.348066406250000000e+03 -1.361441894531250000e+03 -1.311118774414062500e+03 -1.207836059570312500e+03 -1.255661132812500000e+03 -1.390180541992187500e+03 -1.480138061523437500e+03 -1.488689941406250000e+03 -1.452777343750000000e+03 -1.347123413085937500e+03 -1.391256103515625000e+03 -1.523912109375000000e+03 -1.616168457031250000e+03 -1.618919799804687500e+03 -1.580156982421875000e+03 -1.470754516601562500e+03 -1.505686889648437500e+03 -1.613589965820312500e+03 -1.704973876953125000e+03 -1.688956054687500000e+03 -1.573204467773437500e+03 -1.455695678710937500e+03 -1.444384033203125000e+03 -1.559789794921875000e+03 -1.684836181640625000e+03 -1.667870117187500000e+03 -1.537477905273437500e+03 -1.424764770507812500e+03 -1.358997802734375000e+03 -1.330511962890625000e+03 -1.439275634765625000e+03 -1.407359619140625000e+03 -1.271296508789062500e+03 -1.168838500976562500e+03 -1.099422241210937500e+03 -1.110330444335937500e+03 -1.233920043945312500e+03 -1.220491455078125000e+03 -1.089476318359375000e+03 -9.827055053710937500e+02 -9.037623291015625000e+02 -9.565855102539062500e+02 -1.101200073242187500e+03 -1.134070922851562500e+03 -1.040683349609375000e+03 -9.371177368164062500e+02 -8.267092285156250000e+02 -8.320657348632812500e+02 -9.704306640625000000e+02 -1.046510131835937500e+03 -1.002187927246093750e+03 -9.051909790039062500e+02 -7.761748046875000000e+02 -7.314393920898437500e+02 -8.441818847656250000e+02 -9.268428344726562500e+02 -8.851293334960937500e+02 -7.875056152343750000e+02 -6.611007080078125000e+02 -6.263524169921875000e+02 -7.229144897460937500e+02 -7.884291381835937500e+02 -7.184367675781250000e+02 -6.155765380859375000e+02 -4.920034179687500000e+02 -4.648402709960937500e+02 -5.685046997070312500e+02 -6.360821533203125000e+02 -5.683140869140625000e+02 -4.856939697265625000e+02 -3.765409545898437500e+02 -3.269724731445312500e+02 -4.165061645507812500e+02 -5.455139160156250000e+02 -5.920473022460937500e+02 -5.351563720703125000e+02 -5.050506591796875000e+02 -4.158280029296875000e+02 -4.035895996093750000e+02 -4.892374572753906250e+02 -5.956074218750000000e+02 -5.445175781250000000e+02 -6.146407470703125000e+02 -7.571049804687500000e+02 -8.278371582031250000e+02 -8.778640136718750000e+02 -9.433559570312500000e+02 -8.524914550781250000e+02 -9.108256835937500000e+02 -9.818370971679687500e+02 -1.112082519531250000e+03 -1.154799926757812500e+03 -1.221500854492187500e+03 -1.123910400390625000e+03 -1.171636962890625000e+03 -1.287221801757812500e+03 -1.407686279296875000e+03 -1.485389282226562500e+03 -1.583904785156250000e+03 -1.516774291992187500e+03 -1.582924194335937500e+03 -1.727305664062500000e+03 -1.796459594726562500e+03 -1.857154785156250000e+03 -1.947388427734375000e+03 -1.871156250000000000e+03 -1.944188598632812500e+03 -2.084092773437500000e+03 -2.129086425781250000e+03 -2.110864990234375000e+03 -2.114539062500000000e+03 -2.002220703125000000e+03 -2.019240478515625000e+03 -2.137429687500000000e+03 -2.240786132812500000e+03 -2.204388427734375000e+03 -2.109718261718750000e+03 -1.979261230468750000e+03 -1.917582763671875000e+03 -1.870179077148437500e+03 -1.979654052734375000e+03 -1.945834472656250000e+03 -1.819107421875000000e+03 -1.703374267578125000e+03 -1.630077026367187500e+03 -1.596036254882812500e+03 -1.712956787109375000e+03 -1.689370483398437500e+03 -1.554977539062500000e+03 -1.449541503906250000e+03 -1.394499023437500000e+03 -1.433298339843750000e+03 -1.560918457031250000e+03 -1.546395263671875000e+03 -1.416727172851562500e+03 -1.303125000000000000e+03 -1.242662719726562500e+03 -1.289035644531250000e+03 -1.423751098632812500e+03 -1.421499511718750000e+03 -1.301079345703125000e+03 -1.175649780273437500e+03 -1.104284912109375000e+03 -1.114550781250000000e+03 -1.247942016601562500e+03 -1.247094970703125000e+03 -1.137702148437500000e+03 -1.002570983886718750e+03 -9.111639404296875000e+02 -9.315299682617187500e+02 -1.076229370117187500e+03 -1.110891845703125000e+03 -1.019820861816406250e+03 -8.881946411132812500e+02 -7.789893798828125000e+02 -7.334072875976562500e+02 -8.476551513671875000e+02 -9.685853881835937500e+02 -9.874793701171875000e+02 -8.647985839843750000e+02 -7.755928955078125000e+02 -7.135847167968750000e+02 -7.926116333007812500e+02 -9.117517089843750000e+02 -9.622895507812500000e+02 -8.485276489257812500e+02 -8.153274536132812500e+02 -9.134030151367187500e+02 -1.044175415039062500e+03 -1.153083496093750000e+03 -1.166429809570312500e+03 -1.036833251953125000e+03 -9.764837036132812500e+02 -1.039189086914062500e+03 -1.175911376953125000e+03 -1.269595703125000000e+03 -1.240345092773437500e+03 -1.118766479492187500e+03 -1.004267089843750000e+03 -9.576334838867187500e+02 -1.067789306640625000e+03 -1.080347778320312500e+03 -9.809711914062500000e+02 -8.435678710937500000e+02 -7.408353271484375000e+02 -7.441046752929687500e+02 -8.871939086914062500e+02 -9.425516357421875000e+02 -8.729547729492187500e+02 -7.378211669921875000e+02 -6.427541503906250000e+02 -5.446458129882812500e+02 -6.085831298828125000e+02 -7.176790771484375000e+02 -7.556666259765625000e+02 -6.329467773437500000e+02 -6.027039794921875000e+02 -7.025521240234375000e+02 -8.245245361328125000e+02 -9.115313720703125000e+02 -9.675264282226562500e+02 -8.580595703125000000e+02 -8.759863891601562500e+02 -9.594798583984375000e+02 -1.107028808593750000e+03 -1.186203369140625000e+03 -1.244292846679687500e+03 -1.141333984375000000e+03 -1.159739868164062500e+03 -1.281307495117187500e+03 -1.405626953125000000e+03 -1.502776733398437500e+03 -1.562306396484375000e+03 -1.460270507812500000e+03 -1.482982421875000000e+03 -1.608852661132812500e+03 -1.695218017578125000e+03 -1.755507690429687500e+03 -1.800657348632812500e+03 -1.688334228515625000e+03 -1.724945556640625000e+03 -1.814182128906250000e+03 -1.886143554687500000e+03 -1.886710449218750000e+03 -1.853091918945312500e+03 -1.733168212890625000e+03 -1.743273681640625000e+03 -1.832889404296875000e+03 -1.937150878906250000e+03 -1.902097900390625000e+03 -1.807004394531250000e+03 -1.699044189453125000e+03 -1.709036621093750000e+03 -1.794471679687500000e+03 -1.908007324218750000e+03 -1.894587524414062500e+03 -1.873818237304687500e+03 -1.770097412109375000e+03 -1.809826416015625000e+03 -1.901312744140625000e+03 -2.002979614257812500e+03 -2.000402709960937500e+03 -2.003795410156250000e+03 -1.878021118164062500e+03"

In [22]:
len(l.split())

630